In [1]:
# Imports

# for running on wayland, ignore on other platforms #
import os                                           #
os.environ["XDG_SESSION_TYPE"] = "xcb"              #
#####################################################

import cv2
from ultralytics import YOLO
import pandas as pd
from IPython.display import display


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split
import numpy as np


In [25]:
# dataframe generation and class attribute addition

pose_model = YOLO('yolov8s-pose.pt')

main_dir = 'dataset/train/'

position_df = pd.DataFrame()

for class_name in os.listdir(main_dir):
    class_path = os.path.join(main_dir, class_name)
    
    if os.path.isdir(class_path):
        for filename in os.listdir(class_path):
            if filename.endswith(('.jpg', '.jpeg', '.png')):
                image_path = os.path.join(class_path, filename)
                frame = cv2.imread(image_path)

                results = pose_model(frame, device=0)

                x_values = []
                y_values = []
                data = {}
                img_df = pd.DataFrame()

                for result in results:
                    kpts = result.keypoints
                    upperbody_kpts = 11

                    for person in range(len(kpts)):
                        for kp_index in range(upperbody_kpts):
                            keypoint = kpts.xy[person, kp_index]
                            x, y = int(keypoint[0].item()), int(keypoint[1].item())

                            x_values.append(x)
                            y_values.append(y)

                        for i in range(len(x_values)):
                            data[f'x_{i}'] = x_values[i]
                            data[f'y_{i}'] = y_values[i]

                        x_values = []
                        y_values = []

                        # Add a column for class (0 for "healthy", 1 for "risk")
                        data['class'] = 0 if class_name == 'healthy' else 1

                        temp_df = pd.DataFrame([data])
                        img_df = pd.concat([img_df, temp_df], ignore_index=True)
                        temp_df = {}

                # Concatenate per-image DataFrame to the main DataFrame
                position_df = pd.concat([position_df, img_df], ignore_index=True)

# Display the resulting DataFrame
display(position_df)



0: 448x640 1 person, 11.3ms
Speed: 1.6ms preprocess, 11.3ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 10.7ms
Speed: 1.4ms preprocess, 10.7ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)

0: 384x640 2 persons, 9.1ms
Speed: 1.1ms preprocess, 9.1ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 person, 14.6ms
Speed: 1.6ms preprocess, 14.6ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 416x640 1 person, 11.0ms
Speed: 1.5ms preprocess, 11.0ms inference, 0.9ms postprocess per image at shape (1, 3, 416, 640)

0: 448x640 1 person, 11.6ms
Speed: 1.4ms preprocess, 11.6ms inference, 0.9ms postprocess per image at shape (1, 3, 448, 640)

0: 352x640 1 person, 8.5ms
Speed: 1.3ms preprocess, 8.5ms inference, 0.8ms postprocess per image at shape (1, 3, 352, 640)

0: 320x640 2 persons, 8.5ms
Speed: 1.1ms preprocess, 8.5ms inference, 0.9ms postprocess per image at shape (1, 3, 320, 6

,x_0,y_0,x_1,y_1,x_2,y_2,x_3,y_3,x_4,y_4,...,y_6,x_7,y_7,x_8,y_8,x_9,y_9,x_10,y_10,class
0,478,163,0,0,440,140,0,0,334,189,...,348,678,499,384,509,575,194,509,184,1
1,488,235,528,190,450,188,586,195,399,188,...,393,822,479,56,501,736,267,251,252,1
2,169,43,166,38,0,0,171,27,0,0,...,33,187,74,192,67,153,78,156,77,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,130,111,145,87,106,88,166,106,75,111,...,227,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,201,124,207,117,195,120,217,120,189,126,...,151,0,0,0,0,0,0,0,0,0
424,211,41,204,34,206,45,201,23,205,49,...,39,216,2,251,37,0,0,295,30,0
425,160,157,167,152,155,150,178,154,147,149,...,0,0,0,0,0,0,0,0,0,0
426,88,82,95,86,93,76,0,0,93,68,...,57,32,123,24,43,18,104,6,38,0


In [ ]:
# model trainer

class EmotionModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(EmotionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

class BodyLandmarksDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        print(idx)
        return torch.tensor(self.data[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)
    
def get_class_list(df):
    # Ensure the DataFrame is not empty
    if df.empty:
        print("DataFrame is empty.")
        return None

    # Get the last column name
    last_attribute = df.columns[-1]

    # Extract the values of the last (class) attribute and enter into a list
    values_list = df[last_attribute].tolist()

    return values_list

# 11x2 (x, y) body landmarks (22 features) for each sample
y = get_class_list(position_df)  # Output labels
position_df.drop(position_df.columns[-1], axis=1, inplace=True)
X = position_df  # Position (pandas dataframe)



# Split data into train and test sets using PyTorch's native functionality
dataset = BodyLandmarksDataset(X, y)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Initialize the model, loss function, and optimizer
input_size = 22  # 11x2 coordinates
output_size = 2  # Number of output classes
hidden_size = 64

classifier_model = EmotionModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier_model.parameters(), lr=0.001)

# Training loop
epochs = 10

for epoch in range(epochs):
    for inputs, labels in train_loader:
        print(inputs, labels)
        optimizer.zero_grad()
        outputs = classifier_model(inputs.view(-1, input_size))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Evaluation on the test set
classifier_model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = classifier_model(inputs.view(-1, input_size))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')
